# Detekcia meteorov na rádiových spektrogramoch založená na konvolučných neurónových sieťach

## Príprava pracovného prostredia pre prácu s YOLO

Inštalácia knižníc potrebných pre prácu a klonovanie verzie YOLOv5s.

In [ ]:
!pip install opencv-contrib-python
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Import knižnýc potrebných na vytvorenie modelu.

In [ ]:
import torch
import os
import cv2
import time
import datetime
import csv
import PIL
import tensorflow as tf
import shutil
import pandas as pd
import numpy as np
import json
import math 
import scipy.io
import scipy.misc
from IPython.display import Image, clear_output
from matplotlib.pyplot import imshow
from PIL import Image, ImageDraw
from skimage.io import imread
from skimage.transform import resize
from tensorflow.keras.models import load_model, Model
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from sys import path
from os import listdir
from os.path import isfile, join

## Uloženie len pozitívne označených dát
### Vstupy
data: premenná obsahujúca cestu k súborom ozveny-classifications_3.csv a ozveny-dokoncenie-classifications.csv <br>
pozitivne2: súbor, do ktorého sa uložia len pozitívne anotované obrázky

In [ ]:
data=pd.read_csv("ozveny-classifications_3.csv")
#data=pd.read_csv("ozveny-dokoncenie-classifications.csv")
i=0
ano=0
while (i<len(data)):
        je_ozvena2=json.loads(data.iloc[i,1])[0]["value"]
        if je_ozvena2=="Áno":
            event2=json.loads(data.iloc[i,2])[str(data.iloc[i,3])]["Filename"]
            for root, dirs, files in os.walk(rootPath):
                for file in files:
                    if file == event2:
                        im = Image.open(os.path.join(root, "%s" % (file)))
                        im.save(os.path.join('pozitivne2', "%s" % (file))) 
            ano+=1
        i+=1    
print(ano)   

## Vytvorenie textových súborov pre jednotlivé množiny
### Vstupy
cesta_tren: cesta k súboru s dátami na trénovanie, ktorý sme vytvorili ručne <br>
cesta_valid: cesta k súboru s dátami na validáciu, ktorý sme vytvorili ručne <br>
cesta_test: cesta k súboru s dátami na testovanie, ktorý sme vytvorili ručne

Vytvorenie textového súboru pre trénovaciu množinu.

In [ ]:
cesta_tren=os.getcwd()+"/trenovacia"              
with open("trenovacia.txt", "w") as a:
    for path, subdirs, files in os.walk(cesta_tren):
        for filename in files:
            f = os.path.join(path, filename)
            a.write(str(f) + os.linesep) 

Vytvorenie textového súboru pre validačnú množinu.

In [ ]:
cesta_valid=os.getcwd()+"/validacna"               
with open("validacna.txt", "w") as a:
    for path, subdirs, files in os.walk(cesta_valid):
        for filename in files:
            f = os.path.join(path, filename)
            a.write(str(f) + os.linesep) 

Vytvorenie textového súboru pre testovaciu množinu.

In [ ]:
cesta_test=os.getcwd()+"/testovacia"                        
with open("testovacia.txt", "w") as a:
    for path, subdirs, files in os.walk(cesta_test):
        for filename in files:
            f = os.path.join(path, filename)
            a.write(str(f) + os.linesep) 

## Vytvorenie YAML súboru
### Vstupy
train: cesta k textovému súboru trénovacích dát <br>
test: cesta k textovému súboru testovacích dát <br>
val: cesta k textovému súboru validačných dát <br>
nc: počet tried, ktoré model bude rozlišovať <br>
names: názvy jednotlivých tried zaradom 

#### Príklad YAML súboru
train: /home/jovyan/data/lightning/Kamila/trenovacia.txt <br>
test: /home/jovyan/data/lightning/Kamila/testovacia.txt <br>
val: /home/jovyan/data/lightning/Kamila/validacna.txt <br>
nc: 4 <br>
names: ['U bez headecha','U s headechom','O bez headecha','O s headechom']

## Trénovanie modelu
### Vstupy
cesta pre spustenie trénovania: yolov5/train.py <br>
cesta k yaml súboru vstupných dát: /home/jovyan/data/lightning/Kamila/ozveny.yaml <br>
cesta k yaml súboru modelu yolov5s: /home/jovyan/data/lightning/Kamila/yolov5/models/yolov5s.yaml

In [ ]:
!python yolov5/train.py --img 609 --rect --batch 64 --epochs 400 --data /home/jovyan/data/lightning/Kamila/ozveny.yaml --cfg /home/jovyan/data/lightning/Kamila/yolov5/models/yolov5s.yaml --save-period 40 --workers 0

## Detekcia modelu
### Vstupy
cesta pre spustenie testovania: yolov5/train.py <br>
cesta k testovacím dátam: /home/jovyan/data/lightning/Kamila/testovacia <br>
cesta k jednému zo súborov z trénovania: yolov5/runs/train/exp16/weights/best

In [ ]:
!python yolov5/detect.py --source /home/jovyan/data/lightning/Kamila/testovacia --weights yolov5/runs/train/exp16/weights/best.pt --iou 0.3 --conf 0.24 --img 609 --save-txt --save-conf

## Vytvorenie výstupnej tabuľky 
### Vstupy
img_cesta: cesta k súboru s obrázkami detekcií <br>
tab_cesta: cesta k súboru s textovými informáciami o detekciách <br>
csvOut: premenná s názvom .csv súboru <br>
path: premenná s cestou k informáciam o detekciách

In [ ]:
img_cesta=os.getcwd()+"/yolov5/runs/detect/exp34"
tab_cesta=os.getcwd()+"/yolov5/runs/detect/exp34/labels"
csvOut='outputcsv34_2.csv'
path='yolov5/runs/detect/exp34/labels'
name=[f for f in listdir(tab_cesta) if isfile(join(tab_cesta, f))]
with open(csvOut, mode='w') as csv_file:
    fieldnames=['event name','label','x','y','width','height','confidence','length_in_sec','real_begins_in_sec','real_begins_in_time','picture_ends_in','date']
    writer=csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for root, dirs, files in os.walk(path):
        for name in files:
            file=path+"/"+name
            with open(file) as f:
                meno=os.path.splitext(name)[0]
                lines=f.readlines()
                dlzka=len(lines)
                i=0
                while i<dlzka:
                    try:
                        image1=Image.open(os.path.join(img_cesta, name.split(".")[0]+".jpg"))
                        image2=(os.path.join(img_cesta, name.split(".")[0])).split("/")[10].split("_")[1]
                        image3=(os.path.join(img_cesta, name.split(".")[0])).split("/")[10].split("_")[0]
                    except:
                        break
                    line=lines[i]
                    splitted = line.split(" ")
                    list(map(lambda x:x.strip(),splitted))
                    length=round((float(splitted[3])*(image1.size[0])*(5/168.5)),4)        
                    real_beg=round((float(splitted[1])-(float(splitted[3]))/2)*(image1.size[0])*(5/168.5),4)
                    begins=image2[0:2]+":"+image2[2:4]+":"+image2[4:6]
                    t = datetime.time(int(image2[0:2]),int(image2[2:4]),int(image2[4:6]))
                    date=image3[11:]+"."+image3[9:11]+"."+image3[5:9]
                    od_zaciatku=str(real_beg)
                    premena=od_zaciatku.split(".")
                    sekundy3=int(premena[0])
                    stotiny3=premena[1]
                    ts = t.hour * 3600 + t.minute * 60 + t.second
                    a=int(ts-35+sekundy3)
                    conversion = datetime.timedelta(seconds=a)
                    converted_time=str(conversion)+":"+str(stotiny3)[0:3]
                    if(int(splitted[0])==0):
                        a="krátka bez headecha"
                    if(int(splitted[0])==1):
                        a="krátka s headechom"
                    if(int(splitted[0])==2):
                        a="dlhá bez headecha"
                    if(int(splitted[0])==3):
                        a="dlhá s headechom"
                    writer.writerow({
                        'event name':meno,
                        'label':a,                 
                        'x':splitted[1], 
                        'y':splitted[2],
                        'width':splitted[3],
                        'height':splitted[4],
                        'confidence':splitted[5],
                        'length_in_sec':length,
                        'real_begins_in_sec':real_beg,
                        'real_begins_in_time':converted_time,
                        'picture_ends_in':begins,
                        'date':date})
                    i=i+1
